In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import random 
import tensorflow as tf
import json 
#impoort the split 
from sklearn.model_selection import train_test_split
import numpy as np
#import keras classifier   
#IMPORY SGD 
from tensorflow.keras.optimizers import SGD


In [3]:

with open("/home/mohammad/Desktop/First_AId-1.0-sequence-to-sequence-/intents (2).json") as file:
    data = json.load(file)   


In [4]:
import numpy as np

# Define preprocessing functions
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

# Initialize lists for words, labels, training data, and output
words = []
labels = []
training_data = []
output_data = []

# Tokenize and stem each pattern, and prepare training data and output
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = tokenize(pattern)
        words.extend(tokens)
        training_data.append(tokens)
        output_data.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

# Stem and lowercase words, remove duplicates, and sort
words = sorted(list(set([stem(word) for word in words if word != "?"])))
labels = sorted(labels)

# Initialize empty output rows
out_empty = [0] * len(labels)

# Prepare training and output arrays
X = []
y = []

for idx, doc in enumerate(training_data):
    bag = []

    # Stem and lowercase words in the pattern
    tokens = [stem(word) for word in doc]

    # Create bag of words
    for word in words:
        bag.append(1) if word in tokens else bag.append(0)

    # Create output row
    output_row = list(out_empty)
    output_row[labels.index(output_data[idx])] = 1

    X.append(bag)
    y.append(output_row)

# Conve
training = np.array(X)
output = np.array(y)




In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Reset the default graph
tf.reset_default_graph()

# Define the model
model = tf.keras.Sequential([
    # Input layer (implicit by specifying input_shape in the first Dense layer)
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(training[0]),)),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(output[0]), activation='softmax')
])

# Define optimizer
sgd = tf.train.MomentumOptimizer(learning_rate=0.01, momentum=0.9, use_nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )    

# Fit the model
model.fit(training, output, epochs=350, batch_size=5, verbose=1)


Instructions for updating:
non-resource variables are not supported in the long term
Train on 1345 samples
Epoch 1/350
1345/1345 [==============================] - 0s 172us/sample - loss: 6.1152 - acc: 0.0156
Epoch 2/350
1345/1345 [==============================] - 0s 111us/sample - loss: 5.6530 - acc: 0.0320
Epoch 3/350
1345/1345 [==============================] - 0s 110us/sample - loss: 5.2856 - acc: 0.0662
Epoch 4/350
1345/1345 [==============================] - 0s 111us/sample - loss: 4.8907 - acc: 0.1227
Epoch 5/350
1345/1345 [==============================] - 0s 110us/sample - loss: 4.5056 - acc: 0.1717
Epoch 6/350
1345/1345 [==============================] - 0s 110us/sample - loss: 4.0537 - acc: 0.2097
Epoch 7/350
1345/1345 [==============================] - 0s 108us/sample - loss: 3.6136 - acc: 0.2751
Epoch 8/350
1345/1345 [==============================] - 0s 109us/sample - loss: 3.1865 - acc: 0.3175
Epoch 9/350
1345/1345 [==============================] - 0s 110us/sample - lo

In [6]:
model 

In [7]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [8]:
def chat(string1):

        # Preprocess input text into bag of words
        inp_bag_of_words = bag_of_words(string1, words)

        # Reshape input for model prediction
        inp_bag_of_words = inp_bag_of_words.reshape(1, inp_bag_of_words.shape[0])

        # Predict using the model
        results = model.predict(inp_bag_of_words)[0]

        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.3:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            return (random.choice(responses)+ "/n")
            

        else:
            return ("I didn't get that, try again")


In [9]:
##save the model     
model.save('chatbot_model.h5') 

/home/mohammad/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget, QTextEdit, QPushButton, QHBoxLayout
from PyQt5.QtGui import QIcon  
import emoji
from PyQt5.QtCore import QTimer, QPropertyAnimation, Qt  
class ChatGUI(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("First AId")  

        topLayout = QVBoxLayout()
        outerLayout = QVBoxLayout()
        bottomLayout = QHBoxLayout()

        self.label = QLabel(emoji.emojize("      First AI 💊", variant="emoji_type" )+'d Chatbot')  
        self.label.setFixedSize(350, 50)
        self.inputTextEdit = QTextEdit()  
        self.inputTextEdit.setFixedSize(300, 50)     
        self.inputTextEdit.setPlaceholderText("Type your message here...")  # Set placeholder text
        self.outputTextEdit = QTextEdit()   
        self.outputTextEdit.setReadOnly(True)   
        self.outputTextEdit.setFixedSize(350, 300)

        self.sendButton = QPushButton("")  
        self.sendButton.setIcon(QIcon("send.png"))  
        self.sendButton.setFixedSize(50, 50)     
        self.sendButton.setObjectName("sendButton")   

        topLayout.addWidget(self.label)
        bottomLayout.addWidget(self.inputTextEdit)  
        bottomLayout.addWidget(self.sendButton)
        outerLayout.addLayout(topLayout)  # Add topLayout to outerLayout
        outerLayout.addWidget(self.outputTextEdit)
        outerLayout.addLayout(bottomLayout)
        disclaimer = QLabel("  get well soon... Made with ❤️ by First AI 💊d Chatbot Team")
        disclaimer.setStyleSheet("font-size: 12px; color: #888;")
        outerLayout.addWidget(disclaimer)  
       
        def animateDisclaimer(self):
            animation = QPropertyAnimation(self.disclaimer, b"rotation")
            animation.setDuration(1000)
            animation.setStartValue(0)
            animation.setEndValue(5)
            animation.setEasingCurve(Qt.CurveType.InOutQuad)
            animation.start()
            self.disclaimer.setText("Reminder: This chatbot provides First Aid advice only.")
        
        # Set the main layout for the window
        self.setLayout(outerLayout)

        # Connect button click to function
        self.sendButton.clicked.connect(self.processText)

        # Apply CSS styling
        self.setStyleSheet("""
            QWidget {
                background-color: #000000; /* Dark black background */
                font-family: Arial, sans-serif;
                color: #FFFFFF; /* White text */
            }
            QLabel {
                background-color: #000000;
                border: 2px solid #FFFFFF; /* White border */
                border-radius: 10px;
                padding: 10px;
                box-shadow: 0 4px 8px rgba(255, 255, 255, 0.2); /* White box shadow */
                transition: all 0.3s ease;
                font-size: 24px;
                text-align: center;
                font-weight: bold;
                text-shadow: 2px 2px 4px rgba(255, 255, 255, 0.5); /* White text shadow */
                background: radial-gradient(circle, rgba(0, 0, 0, 0) 0%, rgba(0, 0, 0, 0.5) 100%);
            }
            QLabel:hover {
                transform: scale(1.05);
                box-shadow: 0 8px 16px rgba(255, 255, 255, 0.3);
            }
            QTextEdit {
                background-color: #000000; /* Black background */
                border: 2px solid #FFFFFF;
                border-radius: 5px;
                padding: 8px;
                box-shadow: 0 2px 4px rgba(255, 255, 255, 0.1); /* White box shadow */
                transition: border-color 0.3s ease;
                color: #FFFFFF; /* White text */
            }
            QTextEdit:focus {
                border-color: #FFFFFF; /* White border on focus */
            }
            QPushButton#sendButton {
                background-color: #000000; /* White button */
                border: none;
                border-radius: 5px;
                color: #000000; /* Black text */
                padding: 12px 28px;
                text-align: center;
                text-decoration: none;
                display: inline-block;
                font-size: 16px;
                margin: 4px 2px;
                cursor: pointer;
                border-radius: 5px;
                box-shadow: 0 4px 8px rgba(255, 255, 255, 0.2); /* White box shadow */
                transition: background-color 0.3s, box-shadow 0.2s;
                border-color: #FFFFFF; /* White border */
            }
            QPushButton#sendButton:hover {
                background-color: #45a049;
                box-shadow: 0 6px 12px rgba(255, 255, 255, 0.3);
            }
            QPushButton#sendButton:pressed {
                background-color: #397a3c;
                box-shadow: 0 3px 6px rgba(255, 255, 255, 0.3);
            }
            @media (max-width: 768px) {
                QPushButton {
                    padding: 10px 20px;
                    font-size: 14px;
                }
            }
        """)


    def processText(self):
        input_text = self.inputTextEdit.toPlainText()  
        
        processed_text = chat(input_text)
        self.outputTextEdit.setPlainText(processed_text)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    chat_gui = ChatGUI()
    chat_gui.show()

    sys.exit(app.exec_())


<ipython-input-10-c754920e6660>:6: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  class ChatGUI(QWidget):
/home/mohammad/.local/lib/python3.8/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


SystemExit: 0

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
